In [1]:
!pip install firebase_admin

In [2]:
import cv2
import numpy as np
import time
import geocoder
from collections import deque
from tensorflow.keras.models import load_model
import telepot
#from firebase import firebase
import cv2
import numpy as np
from collections import deque
from keras.models import load_model
import geocoder
import time
import telepot
import firebase_admin
from firebase_admin import credentials, db

bot = telepot.Bot('6457674371:AAE04wSEIjA4pyEvlYeYgqNQ7H0WgRuTiBg')
#db = firebase.FirebaseApplication('https://rvidastech-default-rtdb.firebaseio.com', None)


In [3]:
# Initialize Firebase Admin SDK with your credentials
cred = credentials.Certificate("./serviceAccountKey.json")
firebase_admin.initialize_app(cred, {
    'databaseURL': 'https://rvidastech-default-rtdb.firebaseio.com/'
})

In [4]:
def get_current_location():
    location = geocoder.ip('me')
    location_info = f"LOCATION: {location.city}, {location.state}, {location.country}\n"
    return location_info


In [5]:
def store_data_to_firebase(location_info, timeMoment, video_filename, filename):
    ref = db.reference('violence_alerts')
    ref.push({
        'location': location_info,
        'time': timeMoment,
        'video_url': video_filename,
        'image_url': filename
    })


In [6]:
def send_telegram_alert(location_info, timeMoment, video_filename, filename):
    bot.sendMessage(-1002037531635, f"VIOLENCE ALERT!!\n{location_info}TIME: {timeMoment}")
    with open(video_filename, 'rb') as video_file:
        bot.sendVideo(-1002037531635, video_file)
    bot.sendPhoto(-1002037531635, photo=open(filename, 'rb'))


In [12]:
import cv2
import numpy as np
import time
from keras.models import load_model
from keras.optimizers import Adam

def get_current_location():
    # Dummy implementation for example
    return {"lat": "40.7128 N", "long": "74.0060 W"}

def store_data_to_firebase(location_info, timeMoment, video_filename, filename):
    # Dummy implementation for example
    print("Storing data to Firebase")

def send_telegram_alert(location_info, timeMoment, video_filename, filename):
    # Dummy implementation for example
    print("Sending Telegram alert")

def print_results():
    filename = 'savedImage.jpg'
    video_filename = 'recordedVideo.avi'

    # Get current location
    location_info = get_current_location()

    # Load pre-trained Keras model
    print("Loading model ...")
    model = load_model('./modelnew.h5')

    # Recompile the model if needed
    model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

    # Capture video from webcam
    vs = cv2.VideoCapture(0)

    # Check if webcam is opened successfully
    if not vs.isOpened():
        print("Error: Failed to open webcam.")
        return

    # Get webcam dimensions
    W = int(vs.get(cv2.CAP_PROP_FRAME_WIDTH))
    H = int(vs.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Initialize video writer
    fourcc = cv2.VideoWriter_fourcc(*"MJPG")
    writer = cv2.VideoWriter(video_filename, fourcc, 30, (W, H), True)

    # Loop through video frames
    while True:
        grabbed, frame = vs.read()

        if not grabbed:
            break

        output = frame.copy()

        # Preprocess frame for model input
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = cv2.resize(frame, (128, 128)).astype("float32")
        frame = frame.reshape(128, 128, 3) / 255.0

        # Predict violence probability
        preds = model.predict(np.expand_dims(frame, axis=0))[0]
        label = (preds > 0.50)[0]  # Thresholding predictions

        # Adding text to the output frame
        text = "Violence: {}".format(label)
        text_color = (0, 0, 255) if label else (0, 255, 0)
        cv2.putText(output, text, (35, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.25, text_color, 3)

        # Write frame to output video
        writer.write(output)

        # Display output frame
        cv2.imshow("Output", output)

        # Save image and send alert if violence detected
        if label:
            timeMoment = time.strftime("%H:%M:%S \nDate %d-%m-%y")
            store_data_to_firebase(location_info, timeMoment, video_filename, filename)
            send_telegram_alert(location_info, timeMoment, video_filename, filename)

        # Check for key press to quit
        key = cv2.waitKey(1) & 0xFF
        if key == ord("q"):
            break

    # Clean up
    print("[INFO] cleaning up...")
    if writer is not None:
        writer.release()
    vs.release()
    cv2.destroyAllWindows()

print_results()


Loading model ...


[INFO] cleaning up...
